### Example PySpark Notebook

In [1]:
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
 
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.getOrCreate()


sc=spark.sparkContext

bdf = sc.parallelize([
    Row(label=1.0, weight=1.0, features=Vectors.dense(0.0, 5.0)),
    Row(label=0.0, weight=2.0, features=Vectors.dense(1.0, 2.0)),
    Row(label=1.0, weight=3.0, features=Vectors.dense(2.0, 1.0)),
    Row(label=0.0, weight=4.0, features=Vectors.dense(3.0, 3.0))]).toDF()

blor = LogisticRegression(weightCol="weight")

# Fit the model
lrModel = blor.fit(bdf)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/13 01:13:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/13 01:13:21 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/11/13 01:13:21 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [2]:
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

Coefficients: [-1.2334085664744607,-0.7348970347245963]
Intercept: 3.5651650398595693


In [5]:
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.regression import LassoWithSGD
import numpy as np
data = [
    LabeledPoint(0.0, [0.0]),
    LabeledPoint(1.0, [1.0]),
    LabeledPoint(3.0, [2.0]),
    LabeledPoint(2.0, [3.0])
]
lrm = LassoWithSGD.train(
    sc.parallelize(data), iterations=10, initialWeights=np.array([1.0]))


/opt/conda/envs/taq/lib/python3.11/site-packages/pyspark/mllib/regression.py:540: FutureWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression with elasticNetParam = 1.0. Note the default regParam is 0.01 for LassoWithSGD, but is 0.0 for LinearRegression.
  warnings.warn(


In [6]:
abs(lrm.predict(np.array([0.0])) - 0) < 0.5

abs(lrm.predict(np.array([1.0])) - 1) < 0.5

abs(lrm.predict(SparseVector(1, {0: 1.0})) - 1) < 0.5

abs(lrm.predict(sc.parallelize([[1.0]])).collect()[0] - 1) < 0.5

True